In [ ]:
!pip install mlflow optuna dvc -q
!pip install dvc[s3]
!apt install git -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.9/246.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.

In [16]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.keras
import os
import subprocess
import optuna
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

In [ ]:
MLFLOW_TRACKING_URI = ""
EXPERIMENT_NAME = "AQI hyperparameter & model testing (Manual)"
WINDOW_SIZE = 24
PREDICT_HORIZON = 72  # Predict next 3 days = 72 hours

In [4]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='/home/umair/mlruns/3', creation_time=1752022350617, experiment_id='3', last_update_time=1752022350617, lifecycle_stage='active', name='AQI hyperparameter & model testing (Manual)', tags={}>

In [5]:
!git init
!dvc init
!dvc remote add -d myremote s3://s3-bucket-umairrr

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                  

In [6]:
!git clone https://github.com/uma1r111/10pearls-AQI-Project-
%cd 10pearls-AQI-Project-

Cloning into '10pearls-AQI-Project-'...
remote: Enumerating objects: 404, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 404 (delta 80), reused 145 (delta 49), pack-reused 224 (from 1)
Receiving objects: 100% (404/404), 5.68 MiB | 22.29 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/10pearls-AQI-Project-


In [ ]:
# Load and preprocess data
print("\nPulling latest feature_selection.csv from DVC remote (S3)...")
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_DEFAULT_REGION"] = ""
subprocess.run(["dvc", "pull"], check=True)


Pulling latest feature_selection.csv from DVC remote (S3)...


CompletedProcess(args=['dvc', 'pull'], returncode=0)

In [8]:
for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        print(os.path.join(root, name))

./README.md
./.dvcignore
./requirements.txt
./.gitignore
./full_preprocessed_aqi_weather_data_with_all_features.csv
./karachi_weather_apr1_to_current.csv
./fetch_daily_data.py
./feature_selection.csv.dvc
./feature_selection.csv
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.ipynb
./Exploratory Visualization Analysis/Exploratory Visualization Analysis.txt
./.dvc/config
./.dvc/.gitignore
./.dvc/cache/files/md5/a3/42be088d6bbebcb8002a166df6f909
./.dvc/tmp/btime
./.dvc/tmp/lock
./Data Preprocessing/Data_PreProcessing.ipynb
./Data Preprocessing/data_quality_check.py
./Data Preprocessing/run_preprocessing.py
./Data Collection/karachi_weather_jun2025.ipynb
./Data Collection/pollutants info Apr - Jun.ipynb
./Data Collection/karachi_weather_apr1_jun14.ipynb
./.github/workflows/feature_engineering.yml
./.github/workflows/feature_selection.yml
./.github/workflows/update_data.yml
./Feature Selection/initial_feature_extraction.ipynb
./Feature Selection/feature_selection.py


In [9]:
df = pd.read_csv("feature_selection.csv")

In [10]:
df['datetime'] = pd.to_datetime(df['datetime'])
df = df[(df["datetime"] >= "2025-04-01") & (df["datetime"] <= "2025-07-07")]
df = df.sort_values("datetime")

features = df.drop(columns=["datetime"])
num_features = features.shape[1]  # total features including AQI

In [11]:
print(df["datetime"].head())
print("Filtered Rows:", len(df))

0   2025-04-01 00:00:00
1   2025-04-01 01:00:00
2   2025-04-01 02:00:00
3   2025-04-01 03:00:00
4   2025-04-01 04:00:00
Name: datetime, dtype: datetime64[ns]
Filtered Rows: 2329


# **Sequence Building for Walk forward Validation**

In [12]:
def create_sequences(X, window_size, horizon):
    Xs, ys = [], []
    for i in range(len(X) - window_size - horizon):
        Xs.append(X[i:(i + window_size)].values)
        ys.append(X[(i + window_size):(i + window_size + horizon)].values)
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(features, WINDOW_SIZE, PREDICT_HORIZON)
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

In [13]:
print("Total Samples:", len(features))
print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)

Total Samples: 2329
X_seq shape: (2233, 24, 13)
y_seq shape: (2233, 72, 13)


In [14]:
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_val_flat = y_val.reshape(y_val.shape[0], -1)

# **Optuna** Hyperparameter Tuning

In [17]:
def optuna_objective(trial):
    n_units = trial.suggest_int("n_units", 32, 128)
    n_dense_units = trial.suggest_int("n_dense_units", 32, 128)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    recurrent_dropout = trial.suggest_float("recurrent_dropout", 0.0, 0.3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    epochs = trial.suggest_int("epochs", 30, 50)
    n_lstm_layers = trial.suggest_int("n_lstm_layers", 1, 2)

    model = Sequential()
    model.add(LSTM(n_units, return_sequences=(n_lstm_layers > 1),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   activation=activation,
                   dropout=dropout,
                   recurrent_dropout=recurrent_dropout))

    if n_lstm_layers > 1:
        model.add(LSTM(n_units, activation=activation,
                       dropout=dropout,
                       recurrent_dropout=recurrent_dropout))

    model.add(Dense(n_dense_units, activation=activation))
    model.add(Dense(PREDICT_HORIZON * features.shape[1]))  # output size = 72 x 13 = 936

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse")

    model.fit(X_train, y_train_flat,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(X_val, y_val_flat),
              callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
              verbose=0)

    preds = model.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val_flat, preds))
    return rmse

# Run optimization
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=20)

# Log best trial to MLflow
with mlflow.start_run(run_name="Best_Hyperparameters_MultiOutput"):
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)
    print("Best parameters:", study.best_params)


[I 2025-07-09 22:37:35,864] A new study created in memory with name: no-name-53b9148e-f289-4a7e-88a9-b3da5d8be079
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step


[I 2025-07-09 22:38:05,823] Trial 0 finished with value: 30.49928908192906 and parameters: {'n_units': 97, 'n_dense_units': 55, 'dropout': 0.4753974263563664, 'recurrent_dropout': 0.12743327649369252, 'learning_rate': 0.005068357184179167, 'batch_size': 128, 'activation': 'relu', 'epochs': 32, 'n_lstm_layers': 2}. Best is trial 0 with value: 30.49928908192906.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step


[I 2025-07-09 22:39:57,908] Trial 1 finished with value: 27.410590454875347 and parameters: {'n_units': 114, 'n_dense_units': 75, 'dropout': 0.44408434866593516, 'recurrent_dropout': 0.2278418762233794, 'learning_rate': 0.00024886293208890023, 'batch_size': 32, 'activation': 'tanh', 'epochs': 31, 'n_lstm_layers': 1}. Best is trial 1 with value: 27.410590454875347.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


[I 2025-07-09 23:02:22,574] Trial 18 finished with value: 7.123293404436455 and parameters: {'n_units': 46, 'n_dense_units': 96, 'dropout': 0.21148666894853374, 'recurrent_dropout': 0.12572704532841592, 'learning_rate': 0.009857917679679343, 'batch_size': 128, 'activation': 'tanh', 'epochs': 45, 'n_lstm_layers': 1}. Best is trial 16 with value: 7.122644444815656.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


[I 2025-07-09 23:02:44,928] Trial 19 finished with value: 7.124100073019961 and parameters: {'n_units': 43, 'n_dense_units': 94, 'dropout': 0.2380292730104887, 'recurrent_dropout': 0.1339880892389677, 'learning_rate': 0.008165104889053362, 'batch_size': 128, 'activation': 'tanh', 'epochs': 50, 'n_lstm_layers': 1}. Best is trial 16 with value: 7.122644444815656.


Best parameters: {'n_units': 43, 'n_dense_units': 109, 'dropout': 0.19145830941678826, 'recurrent_dropout': 0.055435392313375666, 'learning_rate': 0.0031867088216470944, 'batch_size': 64, 'activation': 'tanh', 'epochs': 45, 'n_lstm_layers': 1}
🏃 View run Best_Hyperparameters_MultiOutput at: http://172.174.154.85:8000/#/experiments/3/runs/d570976c56294b7c8e2cb7cc7eb1cfa0
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **LSTM Model Fit**

In [18]:
best_params = study.best_params
with mlflow.start_run(run_name="Final_LSTM_MultiOutput_Optuna"):
    model = Sequential()
    model.add(LSTM(best_params["n_units"],
                   return_sequences=(best_params["n_lstm_layers"] > 1),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   activation=best_params["activation"],
                   dropout=best_params["dropout"],
                   recurrent_dropout=best_params["recurrent_dropout"]))

    if best_params["n_lstm_layers"] > 1:
        model.add(LSTM(best_params["n_units"], activation=best_params["activation"],
                       dropout=best_params["dropout"],
                       recurrent_dropout=best_params["recurrent_dropout"]))

    model.add(Dense(best_params["n_dense_units"], activation=best_params["activation"]))
    model.add(Dense(PREDICT_HORIZON * features.shape[1]))

    model.compile(optimizer=Adam(learning_rate=best_params["learning_rate"]), loss="mse")

    model.fit(X_train, y_train_flat,
              epochs=best_params["epochs"],
              batch_size=best_params["batch_size"],
              validation_data=(X_val, y_val_flat),
              callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
              verbose=1)

    preds = model.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val_flat, preds))
    mae = mean_absolute_error(y_val_flat, preds)

    # Log to MLflow
    mlflow.log_params(best_params)
    mlflow.log_metric("final_rmse", rmse)
    mlflow.log_metric("final_mae", mae)
    mlflow.set_tag("model_type", "LSTM_MultiOutput")
    mlflow.set_tag("tuner", "Optuna")
    mlflow.keras.log_model(model, artifact_path="model")

    print("Final model logged to MLflow")
    print("Final RMSE:", rmse)
    print("Final MAE:", mae)

Epoch 1/45


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - loss: 1933.3162 - val_loss: 1563.8527
Epoch 2/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 1420.8428 - val_loss: 1114.3811
Epoch 3/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 1015.9770 - val_loss: 794.9324
Epoch 4/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 750.6372 - val_loss: 564.0725
Epoch 5/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 549.6252 - val_loss: 398.4114
Epoch 6/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 402.0500 - val_loss: 280.5931
Epoch 7/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 290.7694 - val_loss: 199.9519
Epoch 8/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 234.7928 - val_loss: 144.8626
Epoch 9/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 199.5797 - val_loss: 109.5275
Epoch 10/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 168.1226 - val_loss: 86.7835
Epoch 11/45
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 151.5534 - val_loss: 72.6914
Epoch 12/45
28/28 ━━━━━━━━━

2025/07/09 23:05:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/09 23:05:16 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/07/09 23:05:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Final model logged to MLflow
Final RMSE: 7.129881774897387
Final MAE: 3.8864623575769834
🏃 View run Final_LSTM_MultiOutput_Optuna at: http://172.174.154.85:8000/#/experiments/3/runs/53ae4d9b125f4ca9ad69c51d46232f64
🧪 View experiment at: http://172.174.154.85:8000/#/experiments/3


# **Predictions**

In [19]:
last_sequence = features.values[-WINDOW_SIZE:].reshape(1, WINDOW_SIZE, features.shape[1])
future_preds = model.predict(last_sequence).reshape(PREDICT_HORIZON, features.shape[1])
future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
                             periods=PREDICT_HORIZON, freq='H')

future_df = pd.DataFrame(future_preds, columns=features.columns)
future_df["datetime"] = future_dates
future_df.to_csv("future_predictions.csv", index=False)
mlflow.log_artifact("future_predictions.csv")
print(future_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
    aqi_us_lag1  aqi_us_lag12  aqi_us_lag24      pm2_5  log_pm10  \
0     77.452164     77.500862     77.385559  24.382507  4.058289   
1     77.456062     77.470085     77.400368  24.341465  4.074904   
2     77.469391     77.432869     77.410675  24.295153  4.097917   
3     77.489082     77.391174     77.411507  24.269276  4.101819   
4     77.499146     77.373917     77.419403  24.226751  4.100608   
..          ...           ...           ...        ...       ...   
67    78.005913     78.090950     77.837364  24.876987  4.090888   
68    78.031624     78.110245     77.855484  24.906393  4.083210   
69    78.046898     78.134354     77.867790  24.912518  4.082654   
70    78.048653     78.154709     77.884796  24.896120  4.088834   
71    78.048523     78.153297     77.903748  24.877621  4.091031   

    scaled_humidity_%  scaled_temp_C_scaled_log_windspeed_kph   log_so2  \
0            0.018942                                0.696161  2.53793

/tmp/ipython-input-19-489927783.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=df['datetime'].iloc[-1] + pd.Timedelta(hours=1),
